In [1]:
%serialconnect
# PIR pots at top, left: 5V mid: D7=P13  right: G
# looking at PIR pots, left: time, right: sensitivity (clockwise more) 

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py
%sendtofile --source ../Sensor_Kennel/TSL561_funcs.py
%sendtofile --quiet --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 81 lines (2677 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [3]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled       2
pinpir       13

Sent 2 lines (30 bytes) to config.txt.


In [7]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [9]:
%websocketconnect --password wpass ws://192.168.43.186:8266/

** WebSocket connected **
Password: Ready.


In [8]:
%sendtofile main.py

import time, sys, machine
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from BME280_funcs import bme280init, readBME280
from TSL561_funcs import setupTSL561, setregtimings, luminosity, compute_lux
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
pinpir = machine.Pin(int(fconfig["pinpir"]), machine.Pin.IN)

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)
print("topicstem", topicstem)

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

topicbme280 = topicstem+"/bme280"
topicTSL561 = topicstem+"/TSL561"
topicpir = topicstem+"/pir"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"

async def bme280task(client):
    try:
        bme280init(i2c, 0x77)
        await asyncio.sleep_ms(200)
        for n in itertools_count():
            meas = readBME280(0x77)
            if (n%100) == 0:
                print(n, topicbme280, meas)
            if client.isconnected():
                await client.publish(topicbme280, " ".join(map(str, meas)))
            else:
                pinled.value(1-pinled.value())
            await asyncio.sleep_ms(100)
    except OSError as e:
        print("bme280taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(bme280task(client))

async def TSL561task(client):
    try:
        setupTSL561(i2c)
        setregtimings(True, 1)
        for n in itertools_count():
            br, ir = luminosity()
            lux = compute_lux(br, ir)
            if (n%200) == 0:
                print(n, topicTSL561, br, ir, lux)
            if client.isconnected() and lux is not None:
                await client.publish(topicTSL561, "%d %d %f"%(br, ir, lux))
            await asyncio.sleep_ms(60)
    except OSError as e:
        print("TSL561taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(TSL561task(client))
        
async def pirchangedetectiontask(client):
    pirstate = -2
    for n in itertools_count():
        prevpirstate, pirstate = pirstate, pinpir.value()
        if pirstate != prevpirstate:
            if client.isconnected():
                await client.publish(topicpir, str(pirstate), retain=True)
            else:
                pirstate = -2
        blinkstate = ((n%400) == 0)
        pinled.value((1-pirstate) != blinkstate)
        await asyncio.sleep_ms(5)
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(client))
aloop.create_task(TSL561task(client))
aloop.create_task(pirchangedetectiontask(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 101 lines (3545 bytes) to main.py.


In [ ]:
%rebootdevice

In [ ]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled       2
pinpir       13


connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinpir       13


In [ ]:
#%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)
print("topicstem", topicstem)

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

topicscd30 = topicstem+"/scd30"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"

async def SCD30task(client):
    try:
        i2c.writeto(0x61, b"\xd3\x04")   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, b"\x00\x10\x00\x00\x81")   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, b"\x46\x00\x00\x02\xe3")   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, b"\x02\x02")  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == b'\x00\x01':
                y = i2c.writeto(0x61, b"\x03\x00")  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                #crc8(x[:2]) == x[2]  etc for every 3 bytes
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid)
                if client.isconnected():
                    await client.publish(topicscd30, "%f %f %f"%(co2, temp, humid))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD30task(client))
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(SCD30task(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


In [ ]:
import machine, struct, time

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(i2c.scan())
def crc8(b):
    crc = 0xFF
    for c in b:
        crc ^= c
        for i in range(8):
            crc = 0xFF & (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1))
    return crc


i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 


i2c.writeto(0x61, b"\xd3\x04")   # soft reset
i2c.writeto(0x61, b"\x00\x10\x00\x00\x81")   # continuous measurement
i2c.writeto(0x61, b"\x46\x00\x00\x02\xe3")   # 2s measurement interval
scd30

while 1:
    y = i2c.writeto(0x61, b"\x02\x02")  # data ready
    time.sleep_ms(4)
    d = i2c.readfrom(0x61, 2)    # ready is 1
    if d == b'\x00\x01':
        y = i2c.writeto(0x61, b"\x03\x00")  # read data
        time.sleep_ms(4)
        x = i2c.readfrom(0x61, 18)
        #crc8(x[:2]) == x[2]  etc for every 3 bytes
        r = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
        print(r)
    time.sleep_ms(500)
    


